<a href="https://colab.research.google.com/github/oAlanSchmitt/CatalogAI/blob/main/CatalogAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALAÇÃO DO SDK

In [ ]:
!pip install -q -U google-generativeai

IMPORTANDO BIBLIOTECAS

In [ ]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai

CONFIGURAÇÃO DA API_KEY

In [ ]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

CONFIGURAÇÕS DO MODELO

In [85]:
# Configurando a acertividade do modelo
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

# Inicializando o modelo. Vamos utilizar o modelo 'gemini-1.0-pro', por isso não listei todos aqui no código.
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

system_instruction = 'Você só pode recomendar Animes, Series e Filmes. O Conteudo recomendado não pode ser igual ao valor descrito na informação. Jogos e outros tipos de entretenimento estão proibidos. Sua Função: recomendar filme, serie, anime com base nos inputes do usuário. Informação: The Walking Dead, Guerra mundial Z, Zumbilandia, Resposta: Serie:All of us are dead \nSinopse: All of Us Are Dead é uma série sul-coreana da Netflix que conta a história de um grupo de estudantes do ensino médio que ficam presos numa escola enquanto um vírus zumbi se espalha. Os alunos têm de lutar para sobreviver sem comida, água ou comunicação, e para não se transformarem em zumbis. \nNota: 8.  '

DESENVOLVIMENTO DO MODELO

In [93]:
chat = model.start_chat(history = [])

# Mensagem inicial.
display(Markdown("##**Olá, eu sou o CatalogAI**"))
display(Markdown("###*Para começar, responda: *"))

# Perguntas.
ip1 = input('Filme/Anime 1: ')
ip2 = input('Filme/Anime 2: ')
ip3 = input('Filme/Anime 3: ')

# Encaminhando as informações para o modelo trabalhar.
prompt = f'Informação: {ip1}, {ip2}, {ip3}.' '\n' 'Resposta: '

# Gerando resposta com base nas informações especificadas pelo usuário + parametros do sistema definidos anteriormente.
response = chat.send_message(system_instruction + prompt)
print('')
print(response.text.replace('*', ''))








##**Olá, eu sou o CatalogAI**

###*Para começar, responda: *

Filme/Anime 1: Madagascar
Filme/Anime 2: Shrek
Filme/Anime 3: Gato de Botas

Filme: Meu Malvado Favorito
Sinopse: Meu Malvado Favorito é um filme de animação da Universal Pictures que conta a história de Gru, um supervilão que planeia roubar a Lua. No entanto, os seus planos são frustrados por três órfãs, Margo, Edith e Agnes, que ele acaba por adotar. Juntos, eles formam uma família pouco convencional e aprendem a importância do amor e da amizade.
Nota: 7,5.
